In [1]:
from google.cloud import storage
import io

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import skimage
import cv2

import os

# Regression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import RidgeCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

# PCA
from sklearn.decomposition import PCA

# Additional features
from skimage.filters.rank import entropy
from skimage.morphology import disk
from skimage.filters import sobel
from scipy import ndimage as nd

In [2]:
# Replace <BUCKET_NAME> with the name of your bucket
bucket_name = 'biznovate_images'

# Create a client object for the storage service
client = storage.Client()


# Get a reference to the bucket
bucket = client.get_bucket(bucket_name)


In [3]:
# Replace <OBJECT_NAME> with the name of the object you want to read
train_csv = 'train.csv'
test_csv = 'test_masked.csv'


## Load the Train data

In [4]:
# Get a reference to the object
train_blob = bucket.blob(train_csv)
train_csv_contents = train_blob.download_as_string().decode('utf-8')

In [5]:
# Read the string as a pandas dataframe
data_train = pd.read_csv(io.StringIO(train_csv_contents))

In [6]:
data_train.head()

,DHSID_EA,cname,year,lat,lon,n_asset,asset_index,n_water,water_index,cluster_id,adm1dhs,urban,path
0,IA-2015-7-00010004,IA,2015,9.165413,92.742696,22.0,2.650768,22.0,5.000000,10004,1,R,dhs_train/IA-2015-7-00010004.npz
1,IA-2015-7-00010005,IA,2015,8.307356,93.093792,22.0,2.157784,22.0,5.000000,10005,1,R,dhs_train/IA-2015-7-00010005.npz
2,IA-2015-7-00010007,IA,2015,7.016968,93.893226,21.0,1.832751,21.0,4.619048,10007,1,R,dhs_train/IA-2015-7-00010007.npz
3,IA-2015-7-00010016,IA,2015,9.194938,92.800432,22.0,2.746096,22.0,5.000000,10016,1,R,dhs_train/IA-2015-7-00010016.npz
4,IA-2015-7-00010018,IA,2015,8.055606,93.543892,22.0,2.581869,22.0,5.000000,10018,1,R,dhs_train/IA-2015-7-00010018.npz


In [7]:
# X_train df after drop columns
X_train = data_train.drop(['DHSID_EA', 'cname', 'year', 'cluster_id', 'adm1dhs', 'urban', 'path', 'water_index', 'n_water'], axis = 1)
X_train.head(2)

,lat,lon,n_asset,asset_index
0,9.165413,92.742696,22.0,2.650768
1,8.307356,93.093792,22.0,2.157784


In [8]:
# y_train series
y_train = data_train['water_index']
y_train.head(2)

0    5.0
1    5.0
Name: water_index, dtype: float64

## Load the test data

In [9]:
# Get a reference to the object
test_blob = bucket.blob(test_csv)
test_csv_contents = test_blob.download_as_string().decode('utf-8')

In [10]:
# Read the string as a pandas dataframe
data_test = pd.read_csv(io.StringIO(test_csv_contents))

In [11]:
data_test.head()

,DHSID_EA,cname,year,lat,lon,n_asset,asset_index,cluster_id,adm1dhs,urban,path
0,IA-2015-7-00010009,IA,2015,9.220903,92.781530,22.0,2.721812,10009,1,R,dhs_valid/IA-2015-7-00010009.npz
1,IA-2015-7-00010011,IA,2015,7.028410,93.883430,20.0,2.287279,10011,1,R,dhs_valid/IA-2015-7-00010011.npz
2,IA-2015-7-00010017,IA,2015,12.371448,92.783665,22.0,0.677109,10017,1,R,dhs_valid/IA-2015-7-00010017.npz
3,IA-2015-7-00010044,IA,2015,11.727304,92.719257,21.0,1.793683,10044,1,R,dhs_valid/IA-2015-7-00010044.npz
4,IA-2015-7-00010060,IA,2015,9.185310,92.777645,22.0,2.758168,10060,1,R,dhs_valid/IA-2015-7-00010060.npz


In [12]:
# X_train df after drop columns
X_test = data_test.drop(['DHSID_EA', 'cname', 'year', 'cluster_id', 'adm1dhs', 'urban', 'path'], axis = 1)
X_test.head(2)

,lat,lon,n_asset,asset_index
0,9.220903,92.78153,22.0,2.721812
1,7.028410,93.88343,20.0,2.287279


# Basic regression for establishing baseline

In [13]:
# Function to run ridgecv

def ridgecv(X_train, y_train):
    X_tr, X_cv, y_tr, y_cv = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
    
    pipeline = make_pipeline(
    StandardScaler(),
    RidgeCV(alphas=[0.1, 1.0, 10.0, 100.0], cv = 5)
    )

    model = pipeline.fit(X_tr, y_tr)

    print("Optimal alpha: {}".format(model.named_steps['ridgecv'].alpha_))
    print("R^2 score: {:.2f}".format(model.score(X_cv, y_cv)))
    
    return model

In [14]:
# Baseline regression on initial columns

ridgecv(X_train, y_train)

Optimal alpha: 1.0
R^2 score: 0.55


Pipeline(steps=[('standardscaler', StandardScaler()),
                ('ridgecv',
                 RidgeCV(alphas=array([  0.1,   1. ,  10. , 100. ]), cv=5))])

In [15]:
def rf(X_train, y_train):
    X_tr, X_cv, y_tr, y_cv = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
    
    pipeline = make_pipeline(
    StandardScaler(),
    RandomForestRegressor(n_estimators=100, random_state=42)
    )

    model = pipeline.fit(X_tr, y_tr)

    print("R^2 score: {:.2f}".format(model.score(X_cv, y_cv)))
    
    return model

In [16]:
rf(X_train, y_train)

R^2 score: 0.72


Pipeline(steps=[('standardscaler', StandardScaler()),
                ('randomforestregressor',
                 RandomForestRegressor(random_state=42))])

## Converting each image band to features

In [ ]:
# function to read the pixels in band and append as feature to the data_train

def read_featurize(df, band = 0, file=1):
    ''' Flatten specified image and band to numpy array'''
    object_name = df['path'][file]
    
    # Get a reference to the object
    blob = bucket.blob(object_name)
    
    npz_bytes = blob.download_as_bytes()
    img = np.load(io.BytesIO(npz_bytes))['x']
    
    return (img[band,].flatten())

## Flatten the each band and do the PCA on it

In [ ]:
band_num=4
# initialize the feature numpy or the len 255*255
band_feat = np.zeros((data_train.shape[0], 255*255))
test_band_feat = np.zeros((data_test.shape[0], 255*255))

for index in range(data_train.shape[0]):
    band_feat[index] = read_featurize(data_train, band_num, index)

for index in range(data_test.shape[0]):
    test_band_feat[index] = read_featurize(data_test, band_num, index)

# For each band in the image make a feature matrix for all 18K train images
# standardize and transform train 
scaler = StandardScaler()
scaler.fit(band_feat)
band_feat_std = scaler.transform(band_feat)

# transform test
test_band_feat_std = scaler.transform(test_band_feat)

# pca with 3 components fit and transform train
pca = PCA(n_components=n_comp)
pca.fit(band_feat_std)

variable = "band_feat_pca"
result[variable] = pca.transform(band_feat_std)

# pca with 3 components transform test
variable = "test_band_feat_pca"
test_result[variable] = pca.transform(test_band_feat_std)

## Create the feature numpy array

In [ ]:
X_train_np = X_train.to_numpy()
X_test_np = X_test.to_numpy()

X_train_band_pca = np.concatenate((
    X_train_np,
    result['band_feat_pca0'],
    result['band_feat_pca1'],
    result['band_feat_pca2'],
    result['band_feat_pca3'],
    result['band_feat_pca4'],
    result['band_feat_pca5'],
    result['band_feat_pca6'],
    result['band_feat_pca7']
), axis = 1)


X_test_band_pca = np.concatenate((
    X_test_np,
    test_result['test_band_feat_pca0'],
    test_result['test_band_feat_pca1'],
    test_result['test_band_feat_pca2'],
    test_result['test_band_feat_pca3'],
    test_result['test_band_feat_pca4'],
    test_result['test_band_feat_pca5'],
    test_result['test_band_feat_pca6'],
    test_result['test_band_feat_pca7']
), axis = 1)

## Ridge Regression on the feature numpy array

In [ ]:
ridgecv(X_train_band_pca, y_train)

## Random forrest Regression on the feature numpy array

In [ ]:
def rf(X_train, y_train):
    X_tr, X_cv, y_tr, y_cv = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
    
    pipeline = make_pipeline(
    StandardScaler(),
    RandomForestRegressor(n_estimators=100, random_state=42)
    )

    model = pipeline.fit(X_tr, y_tr)

    print("R^2 score: {:.2f}".format(model.score(X_cv, y_cv)))
    
    return model

In [ ]:
rf(X_train_band_pca, y_train)

## Adding Sobel feature, band 5 only

In [17]:
def sobel_filter(df, file=1, band = 4):
    ''' add filter and features the the image only using one band
        Gaussian and Enropy commented out and may be tried if required
    '''
    object_name = df['path'][file]
    
    # Get a reference to the object
    blob = bucket.blob(object_name)
    
    npz_bytes = blob.download_as_bytes()
    img = np.load(io.BytesIO(npz_bytes))['x']
    img = np.uint8(img*255) # changing to uint8
    

    # Entropy image
    # entropy_img = np.round(entropy(img[4,], disk(1)),2)

    # Gaussian image
    # gaussian_img = np.round(nd.gaussian_filter(img[4,], sigma = 1),2)

    # Sobel image
    sobel_img = np.round(sobel(img[band,]),2)
    return sobel_img.flatten()

In [18]:
# Create and write the sobel feature image for each train image

# initialize the feature numpy or the len 255*255
sobel_feat = np.zeros((data_train.shape[0], 255*255))
test_sobel_feat = np.zeros((data_test.shape[0], 255*255))

for index in range(data_train.shape[0]):
    sobel_feat[index] = sobel_filter(data_train, index)


for index in range(data_test.shape[0]):
    test_sobel_feat[index] = sobel_filter(data_test, index)


### PCA for Sobel features band 5

In [19]:
# standardize fit and transform on train
sobel_scaler = StandardScaler().fit(sobel_feat)
sobel_feat_std = sobel_scaler.transform(sobel_feat)

# transform on test
test_sobel_feat_std = sobel_scaler.transform(test_sobel_feat)

# pca fit and transform on train
ncomp = 3
sobel_pca = PCA(n_components=ncomp)
sobel_pca.fit(sobel_feat_std)
sobel_pca_transformed = sobel_pca.transform(sobel_feat_std)

# pca transform on test
test_sobel_pca_transformed = sobel_pca.transform(test_sobel_feat_std)

## Concatenate with sobel features for band 5 

In [20]:
X_train_np = X_train.to_numpy()
X_test_np = X_test.to_numpy()


In [21]:
X_train_band_sobel_pca = np.concatenate((
    X_train_np,
    sobel_pca_transformed
), axis = 1)

X_test_band_sobel_pca = np.concatenate((
    X_test_np,
    test_sobel_pca_transformed
), axis = 1)

## Add entropy filter band 5 only

In [61]:
def entropy_filter(df, file=1, band = 4):
    ''' add entropy filter features on the image only using one band
    '''
    object_name = df['path'][file]
    
    # Get a reference to the object
    blob = bucket.blob(object_name)
    
    npz_bytes = blob.download_as_bytes()
    img = np.load(io.BytesIO(npz_bytes))['x']
    img = np.uint8(img*255) # changing to uint8
    

    # Entropy image
    entropy_img = np.round(entropy(img[band,], disk(1)),2)

    # Gaussian image
    # gaussian_img = np.round(nd.gaussian_filter(img[band,], sigma = 1),2)

    # Sobel image
    # sobel_img = np.round(sobel(img[band,]),2)
    return entropy_img.flatten()

In [62]:
# Create and write the entropy feature image for each train image

# initialize the feature numpy or the len 255*255
entropy_feat = np.zeros((data_train.shape[0], 255*255))
test_entropy_feat = np.zeros((data_test.shape[0], 255*255))

for index in range(data_train.shape[0]):
    entropy_feat[index] = entropy_filter(data_train, index)


for index in range(data_test.shape[0]):
    test_entropy_feat[index] = entropy_filter(data_test, index)

In [64]:
import gc

gc.collect()

2816

In [76]:
del sobel7_pca_transformed

In [77]:
gc.collect()

46

### PCA with entropy on band 5 only

In [70]:
entropy_feat = entropy_feat.astype(np.float16)

In [71]:
test_entropy_feat = test_entropy_feat.astype(np.float16)

In [78]:
# standardize fit and transform on train
entropy_scaler = StandardScaler().fit(entropy_feat)
entropy_feat_std = entropy_scaler.transform(entropy_feat)
entropy_feat_std = entropy_feat_std.astype(np.float16)

# transform on test
test_entropy_feat_std = entropy_scaler.transform(test_entropy_feat)
test_entropy_feat_std = test_entropy_feat_std.astype(np.float16)

# pca fit and transform on train
ncomp = 3
entropy_pca = PCA(n_components=ncomp)
entropy_pca.fit(entropy_feat_std)
entropy_pca_transformed = entropy_pca.transform(entropy_feat_std)

# pca transform on test
test_entropy_pca_transformed = entropy_pca.transform(test_entropy_feat_std)

## Concatenate results from entropy band 5

In [82]:
X_train_sobel_entropy_pca = np.concatenate((
    X_train_band_sobel_pca,
    entropy_pca_transformed
), axis = 1)

X_test_sobel_entropy_pca = np.concatenate((
    X_test_band_sobel_pca,
    test_entropy_pca_transformed
), axis = 1)

## Adding Sobel feature, band 7 only

In [35]:
# # Create and write the sobel feature image for each train image

# # initialize the feature numpy or the len 255*255
# sobel_feat = np.zeros((data_train.shape[0], 255*255))
# test_sobel_feat = np.zeros((data_test.shape[0], 255*255))

# for index in range(data_train.shape[0]):
#     sobel_feat[index] = sobel_filter(data_train, index, band = 6)

# for index in range(data_test.shape[0]):
#     test_sobel_feat[index] = sobel_filter(data_test, index, band = 6)

### PCA with sobel band 7 only

In [44]:
# # standardize fit and transform on train
# sobel_scaler = StandardScaler().fit(sobel_feat)
# sobel_feat_std = sobel_scaler.transform(sobel_feat)

# # transform on test
# test_sobel_feat_std = sobel_scaler.transform(test_sobel_feat)

# # pca fit and transform on train
# ncomp = 3
# sobel_pca = PCA(n_components=ncomp)
# sobel_pca.fit(sobel_feat_std)
# sobel7_pca_transformed = sobel_pca.transform(sobel_feat_std)

# # pca transform on test
# test_sobel7_pca_transformed = sobel_pca.transform(test_sobel_feat_std)

## Concatenate results from Sobel band 7

In [45]:
# X_train_band5_band7_sobel_pca = np.concatenate((
#     X_train_band_sobel_pca,
#     sobel7_pca_transformed
# ), axis = 1)

# X_test_band5_band7_sobel_pca = np.concatenate((
#     X_test_band_sobel_pca,
#     test_sobel7_pca_transformed
# ), axis = 1)

## Regression models

In [83]:
rf(X_train_sobel_entropy_pca, y_train)

R^2 score: 0.71


Pipeline(steps=[('standardscaler', StandardScaler()),
                ('randomforestregressor',
                 RandomForestRegressor(random_state=42))])

## Predictions

In [84]:
pipeline = make_pipeline(
StandardScaler(),
RandomForestRegressor(n_estimators=100, random_state=42)
)

model = pipeline.fit(X_train_sobel_entropy_pca, y_train)

In [86]:
model.predict(X_test_sobel_entropy_pca)

array([4.88489952, 4.6768248 , 4.5311093 , ..., 4.84791988, 4.25078868,
       4.40306912])

In [87]:
y_pred = model.predict(X_test_sobel_entropy_pca)

In [88]:
data_test['DHSID_EA']

0       IA-2015-7-00010009
1       IA-2015-7-00010011
2       IA-2015-7-00010017
3       IA-2015-7-00010044
4       IA-2015-7-00010060
               ...        
2670    IA-2015-7-00360403
2671    IA-2015-7-00360454
2672    IA-2015-7-00360474
2673    IA-2015-7-00360476
2674    IA-2015-7-00360479
Name: DHSID_EA, Length: 2675, dtype: object

In [89]:
final = data_test.assign(water_index = y_pred)

In [90]:
final.columns

Index(['DHSID_EA', 'cname', 'year', 'lat', 'lon', 'n_asset', 'asset_index',
       'cluster_id', 'adm1dhs', 'urban', 'path', 'water_index'],
      dtype='object')

# Exporting the csv file to bucket

In [91]:
contents_csv = final[['DHSID_EA','water_index']].to_csv(index = False)

csv_bytes = bytes(contents_csv, 'utf-8')

blob = bucket.blob('final_sobel_entropy_csv')
blob.upload_from_string(csv_bytes)

### Export feature matrix

In [92]:
# feature_csv = pd.DataFrame(X_train_sobel_entropy_pca).to_csv(index=False)
# csv_bytes = bytes(feature_csv, 'utf-8')
# blob = bucket.blob('feature_sobel_entropy_train_csv')
# blob.upload_from_string(csv_bytes)

In [93]:
# feature_test_csv = pd.DataFrame(X_test_sobel_entropy_pca).to_csv(index=False)
# csv_bytes = bytes(feature_test_csv, 'utf-8')
# blob = bucket.blob('feature_sobel_entropy_test_csv')
# blob.upload_from_string(csv_bytes)